## Implementation of the say can algorithm from google research
The SayCan algorithm tries to map langauage to robot control. SayCan is an algorithm that grounds large language models with robotic affordances for long-horizon planning. Given a set of low-level robotic skills (e.g., "put the green block in the red bowl") and a high-level instruction (e.g., "stack all the blocks"), it scores what a language model believes will help forward the high-level instruction and scores what a robotic affordance model believes is possible. Given a list of low-level and high-level tasks, the LLM will pick tasks that can enable it move forward towards acheiving its goal. 

In [1]:
import openai

# start open AI configuration
openai.api_base = "https://api.naga.ac/v1"
openai.api_key =  "VN7eDdNzbkQkrmEmIr1Gj1Kci3Ed_g6a_atrW14jq6c"

In [ ]:
# install project requirements

# !pip install ftfy regex tqdm fvcore imageio==2.4.1 imageio-ffmpeg==0.4.5
# !pip install git+https://github.com/openai/CLIP.git
# !pip install -U --no-cache-dir gdown --pre
# !pip install pybullet moviepy
# !pip install flax==0.5.3
# !pip install openai
# !pip install easydict
# !pip install imageio-ffmpeg

In [ ]:
# install required libraries
import collections
import datetime
import os
import random
import threading
import time


import cv2  # Used by ViLD.
import clip
from easydict import EasyDict
import flax
from flax import linen as nn
from flax.training import checkpoints
from flax.metrics import tensorboard
import imageio
from heapq import nlargest
import IPython
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from moviepy.editor import ImageSequenceClip
import numpy as np
import openai
import optax
import pickle
from PIL import Image
import pybullet
import pybullet_data
import tensorflow.compat.v1 as tf
import torch
from tqdm import tqdm